In [1]:
import os
import numpy as np 
import pandas as pd
from glob import glob
import cv2
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [2]:
winner_list = pd.DataFrame({'file_path':pd.read_csv(r'..\Paper\Winner_path.csv').iloc[:,1]})
loser_list = pd.DataFrame({'file_path':pd.read_csv(r'..\Paper\Loser_path.csv').iloc[:,1]})

In [3]:
winner_path = '..\Paper\Winner\\'
loser_path = '..\Paper\Loser\\'

In [4]:
winner_list['file_path'] = winner_list['file_path'].apply(lambda path:os.path.join(winner_path, path))
loser_list['file_path'] = loser_list['file_path'].apply(lambda path:os.path.join(loser_path, path))
winner_list['decision'] = 1
loser_list['decision'] = 0

In [5]:
df = pd.concat([winner_list,loser_list]).reset_index(drop=True)

In [9]:
img_list = []
missing = []
for i in range(len(df)):
    img = cv2.imread(df['file_path'].loc[i])
    if img is not None:
        img = cv2.resize(img, (323, 172))
        img_list.append(img)
X = np.concatenate(img_list, axis = 0)

In [10]:
from sklearn.model_selection import train_test_split
import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.applications import VGG16, DenseNet121,VGG19
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large,ResNet152V2
from keras.callbacks import EarlyStopping,LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [12]:
X = X.reshape(len(df), 323,172,3)/255
X_noise = X + np.random.normal(loc=0, scale=0.05, size=X.shape)
X_noise = np.clip(X_noise, 0, 1)

In [13]:
y = df['decision']

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_noise,y,random_state=99)

In [15]:
keras.backend.clear_session()

In [16]:
# Application
input_shape=(323, 172, 3)
inputs = Input(shape=input_shape)

Application = DenseNet121(weights='imagenet', include_top=False)
for layer in Application.layers:
    layer.trainable = False    
app_model = Application(inputs)

# Flatten
x = Dropout(0.2)(app_model)
x = Flatten()(app_model)
# Output
output = Dense(1, activation='sigmoid', name='decision') (x)


model = Model(inputs=[inputs], outputs=[output])


29097984/29084464 [==============================] - 27s 1us/step


In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 323, 172, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, None, None, 1024)  7037504   
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
decision (Dense)             (None, 1)                 51201     
Total params: 7,088,705
Trainable params: 51,201
Non-trainable params: 7,037,504
_________________________________________________________________


In [18]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [19]:
initial_learning_rate = 0.001
def lr_schedule(epoch):
    decay_rate = 0.8
    decay_step = 5  # Adjust this as needed
    return initial_learning_rate * decay_rate ** (epoch // decay_step)
optimizer = Adam(learning_rate=initial_learning_rate)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [20]:
model.compile(loss = 'binary_crossentropy',
             optimizer = optimizer,
             metrics = ['accuracy'])

In [23]:
train = model.fit(X_train, y_train, validation_data = (X_val,y_val),epochs = 40,shuffle=True,callbacks=[lr_scheduler,early_stop])

Epoch 1/40
11/11 [==============================] - 18s 771ms/step - loss: 1.1494 - accuracy: 0.5698 - val_loss: 0.7406 - val_accuracy: 0.5641
Epoch 2/40
11/11 [==============================] - 1s 133ms/step - loss: 0.5090 - accuracy: 0.7521 - val_loss: 0.5499 - val_accuracy: 0.7179
Epoch 3/40
11/11 [==============================] - 1s 133ms/step - loss: 0.4080 - accuracy: 0.7778 - val_loss: 0.5002 - val_accuracy: 0.7521
Epoch 4/40
11/11 [==============================] - 1s 133ms/step - loss: 0.3075 - accuracy: 0.8661 - val_loss: 0.4976 - val_accuracy: 0.7778
Epoch 5/40
11/11 [==============================] - 1s 133ms/step - loss: 0.2439 - accuracy: 0.9202 - val_loss: 0.4466 - val_accuracy: 0.8120
Epoch 6/40
11/11 [==============================] - 1s 133ms/step - loss: 0.2048 - accuracy: 0.9487 - val_loss: 0.5589 - val_accuracy: 0.7350
Epoch 7/40
11/11 [==============================] - 1s 133ms/step - loss: 0.1664 - accuracy: 0.9487 - val_loss: 0.4085 - val_accuracy: 0.8547
Epoch

In [24]:
model.save('model_V2.h5')

# Save the training history
joblib.dump(train.history, 'history_V2.joblib')

['history_V2.joblib']